In [1]:
#simple bert classifier

In [2]:
# !pip install transformers
# !pip install torch
# !pip install pytorch-lightning
# !pip install accelerate -U


In [3]:
#  !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [4]:
import torch

# If there's a GPU available...
torch.cuda.is_available()    


True

In [5]:
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments


In [6]:
df=pd.read_json('yt_training_set.json')

df = pd.DataFrame(df)
# print(df)

#drop missing values
columns_to_dropna = ["title", "channel_title", "view_count", "tags", "description", "label"]

# dropna method drop rows with missing values 
filtered_data = df.dropna(subset=columns_to_dropna, how='any')

# print(filtered_data)
new_df=pd.DataFrame(filtered_data)



new_dataframe= pd.DataFrame({'title': new_df["title"].values,"channel_title": new_df["channel_title"].values,"view_count":new_df["view_count"].values,"tags":new_df["tags"].values,"description":new_df["description"].values,'label': new_df["label"].values})
# print(new_dataframe)
new_df=new_dataframe


# Convert the data to a pandas DataFrame
df_concat = pd.DataFrame(columns=["text", "label"])


for entry in new_df.index:
    
    title=new_df["title"].get(entry) 
    # print(title)
    
    channel_title=new_df["channel_title"].get(entry)
    
    tags=" ".join(new_df["tags"].get(entry ,[]))
    description=new_df["description"].get(entry)
    
    label=new_df["label"].get(entry)

    
    

    concat_text = f"{title} {channel_title} {tags} {description}"
    # concat_text=re.sub("(http\S+)", "", concat_text)
    
    new_row = pd.DataFrame({"text": [concat_text], "label": [label]})
    df_concat = pd.concat([df_concat, new_row], ignore_index=True)
    
df_concat['label'] = df_concat['label'].map({-1: 0, 1: 1})



In [7]:

# Split the data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df_concat['text'].tolist(), df_concat['label'].tolist(), test_size=0.2)


In [8]:

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)




In [9]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    

# Create the datasets
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)




In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Initialization of BERT 
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)  # Adjust num_labels based on your task

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,  # The learning_rate parameter
)

# Create the trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


# Evaluate the model
eval_results = trainer.evaluate()

print(eval_results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/75 [00:00<?, ?it/s]

{'train_runtime': 1128.4376, 'train_samples_per_second': 2.122, 'train_steps_per_second': 0.066, 'train_loss': 0.618732655843099, 'epoch': 3.0}


TrainOutput(global_step=75, training_loss=0.618732655843099, metrics={'train_runtime': 1128.4376, 'train_samples_per_second': 2.122, 'train_steps_per_second': 0.066, 'train_loss': 0.618732655843099, 'epoch': 3.0})

In [17]:
# Save the model

# Specify the path
model_path = "models"
tokenizer_path = "models"

# Save the model and tokenizer

tokenizer.save_pretrained(tokenizer_path)

trainer.save_model(model_path)




In [18]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
# Evaluate the model
eval_results = trainer.evaluate()

print(eval_results)


  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.5858001112937927, 'eval_runtime': 23.8513, 'eval_samples_per_second': 8.385, 'eval_steps_per_second': 1.048, 'epoch': 3.0}
